In [120]:
import pandas as pd

In [121]:
PATH_TRAIN = 'dev.bio'

In [122]:
def read_iob2_file(path):
    data = []
    current_words = []
    current_tags = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue
            tok = line.split('\t')

            current_words.append(tok[1])
            current_tags.append(tok[2])
        else:
            if current_words:
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []

    if current_tags != []:
        data.append((current_words, current_tags))

    df = pd.DataFrame(data, columns=['words', 'tags'])
    df['id'] = df.index
    df = df[['id', 'words', 'tags']]
    
    return df

In [115]:
def read_bio_file(path):
    
    data = []
    current_words = []
    current_tags = []

    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()[2:]
        
    for line in lines:
        
        line = line.strip()
        
        if line: # if line is not an empty line
            tok = line.split('\t')
            current_words.append(tok[0])
            current_tags.append(tok[3])
            
        else:
            if current_words:
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []
            
            
    if current_tags != []:
        data.append((current_words, current_tags))

    df = pd.DataFrame(data, columns=['words', 'tags'])
    df['id'] = df.index
    df = df[['id', 'words', 'tags']]
    
    return df

In [123]:
dev_data = read_bio_file(PATH_TRAIN)

In [117]:
data

,id,words,tags
0,0,"[RT, @USER2362, :, Farmall, Heart, Of, The, Ho...","[O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, O,..."
1,1,"[#Volunteers, are, key, members, of, #CHEO’s, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,2,"[@USER2092, is, n't, it, funny, how, that, alw...","[O, O, O, O, O, O, O, O, O, O, O]"
3,3,"[RT, @USER80, :, Silence, is, better, than, li...","[O, O, O, O, O, O, O, O, O]"
4,4,"[I, just, spent, twenty, minutes, trying, to, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
1634,1634,"[RT, @USER1701, :, FT, ISLAND, -, I, Hope, (, ...","[O, O, O, O, B-PER, O, B-MISC, I-MISC, O, O, O..."
1635,1635,"[@USER1321, @USER2526, Probably, ., He, is, n'...","[O, O, O, O, O, O, O, O, O, O, O, O]"
1636,1636,"[RT, @USER1920, :, @USER1260, @USER2624, it, '...","[O, O, O, O, O, O, O, O, O]"
1637,1637,"[You, have, that, right, ,, nor, do, they, int...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [124]:
import random

def swap_neighboring_letters(word):
    if len(word) <= 1:
        return word  # Return the word unchanged if it has only one character

    # Convert the word into a list of characters for easier manipulation
    word_list = list(word)

    # Choose a random index to swap with its neighboring letter
    idx = random.randint(0, len(word) - 2)  # Ensure that the chosen index is not the last character

    # Swap the character at the chosen index with its neighboring letter
    word_list[idx], word_list[idx + 1] = word_list[idx + 1], word_list[idx]

    # Convert the list of characters back to a string
    return ''.join(word_list)

def swap_letters_in_sentences(df, perc_sent, perc_words, apply_to_all=True):
    random.seed(10)
    df_copy = df.copy(deep=True)
    num_of_sent = int(perc_sent*len(df_copy))
    print(f"Number of sentences to alter: {num_of_sent}")

    #choose rand sentence ids
    sentences_ids = random.sample(range(len(df_copy)), num_of_sent)
    print("type", type(sentences_ids))
    print("len", len(sentences_ids))
    for sentence_id in sentences_ids:
        #indexing the sentence
        sentence = (df_copy["words"][sentence_id]).copy()
        print("sentence type", type(sentence))
        num_words = int(perc_words*len(sentence)) #how many words to perturb
        print("num of w", num_words)
        if num_words > 0:
            words_ids = random.sample(range(len(sentence)), num_words)
            for word in words_ids:
                word_to_change = sentence[word]
                sentence[word]=swap_neighboring_letters(word_to_change)
        df_copy["words"][sentence_id] = sentence

    return num_of_sent, sentences_ids, df_copy

# Swap 20% of sentences, swapping 20% of words in each selected sentenc

In [125]:
n, ids, df_pert = swap_letters_in_sentences(dev_data, 0.01, 1.0)
print(n)
print(ids)
for i in ids:
    print("###########################")
    print(df_pert["words"][i])
    print(dev_data["words"][i])
    if df_pert["words"][i]==dev_data["words"][i]:
        print("SAME!")

Number of sentences to alter: 7
type <class 'list'>
len 7
sentence type <class 'list'>
num of w 14
sentence type <class 'list'>
num of w 3
sentence type <class 'list'>
num of w 17
sentence type <class 'list'>
num of w 23
sentence type <class 'list'>
num of w 25
sentence type <class 'list'>
num of w 18
sentence type <class 'list'>
num of w 15
7
[585, 33, 439, 494, 591, 15, 211]
###########################
['Chnuky', 'braeds', 'form', '@USER190', 'aer', 'execllent', 'fro', 'tearnig', 'nad', 'dunikng', 'itno', 'suop', '🍞', 'UR5L96']
['Chunky', 'breads', 'from', '@USER109', 'are', 'excellent', 'for', 'tearing', 'and', 'dunking', 'into', 'soup', '🍞', 'URL596']
###########################
['U@SER331', 'koaay', ',']
['@USER331', 'okaay', ',']
###########################
['TR', 'U@SER996', ':', 'eH', 's’', 'gto', 'teh', 'Norht', '.', 'Nroth', 'ogt', 'ihs', 'bakc', '.', '#BrignYourGame', 'RUL1417', 'RUL1380']
['RT', '@USER996', ':', 'He', '’s', 'got', 'the', 'North', '.', 'North', 'got', 'his',

/var/folders/ph/wdjw3wn10c1frx2v062_8sbh0000gn/T/ipykernel_89699/4057977354.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy["words"][sentence_id] = sentence
/var/folders/ph/wdjw3wn10c1frx2v062_8sbh0000gn/T/ipykernel_89699/4057977354.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy["words"][sentence_id] = sentence
/var/folders/ph/wdjw3wn10c1frx2v062_8sbh0000gn/T/ipykernel_89699/4057977354.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret